# Prep Upper Colorado use-case.

This a subset of the NHM prepped from a Bandit pull by Parker Norton.  Using this as a
test case for forecasting.

In [2]:
import geopandas as gpd
import requests
import pandas as pd
import rioxarray
from pathlib import Path
import numpy as np

In [2]:
root_dir = '/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/'

In [3]:
root_dir

'/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/'

In [4]:
def read_param_values(dir, field):
    filename = Path(dir) / "myparam.param"
    values = []  # List to store the elevation values
    start_collecting = False  # Flag to start collecting values
    count_values = 0  # To count the values read
    
    with open(filename, 'r') as file:
        for line in file:
            stripped_line = line.strip()  # Remove any leading/trailing whitespace
            
            if stripped_line == field:
                # Skip the next four lines after 'hru_elev'
                for _ in range(4):
                    next(file)
                start_collecting = True  # Set flag to start collecting after skipping
                continue
            
            if stripped_line == '####' and start_collecting:
                break  # Stop reading if we hit #### after starting to collect
            
            if start_collecting:
                try:
                    # Convert the line to a float and add to the list
                    value = float(stripped_line)
                    values.append(value)
                    count_values += 1
                except ValueError:
                    continue  # If conversion fails, skip the line
        print(f"Read {count_values} {field} values")
    return np.array(values)

In [5]:
gdf = gpd.read_file(root_dir + "GIS/model_nhru.shp")
gdf

,nhm_id,model_hru_,geometry
0,83301,274,"POLYGON ((-1324125.133 1609094.887, -1324275.0..."
1,83302,275,"POLYGON ((-1331914.568 1623225.112, -1331901.0..."
2,83359,332,"POLYGON ((-1308375.036 1622938.630, -1308465.2..."
3,83364,337,"POLYGON ((-1337655.210 1640205.133, -1337535.3..."
4,83366,339,"POLYGON ((-1339634.862 1640985.080, -1339695.0..."
...,...,...,...
3846,86611,3556,"POLYGON ((-1256384.842 2032365.123, -1256295.1..."
3847,86615,3560,"POLYGON ((-1255034.951 2041934.777, -1255154.7..."
3848,86686,3631,"POLYGON ((-1244805.129 1970954.938, -1244744.9..."
3849,86691,3636,"POLYGON ((-1255664.885 1971375.143, -1255544.7..."


In [6]:
# Find invalid geometries
invalid_geometries = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

# Correct invalid geometries by explicitly targeting the original DataFrame
gdf.loc[~gdf.is_valid, 'geometry'] = gdf.loc[~gdf.is_valid, 'geometry'].apply(lambda geom: geom.buffer(0))

# Re-check for invalid geometries after the correction
invalid_geometries_after = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries_after)} invalid geometries after correction.")

Found 177 invalid geometries.
Found 0 invalid geometries after correction.


In [7]:
gdf.crs

<Projected CRS: ESRI:102039>
Name: USA_Contiguous_Albers_Equal_Area_Conic_USGS_version
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
gdf = gdf.to_crs("EPSG:4326")
gdf

,nhm_id,model_hru_,geometry
0,83301,274,"POLYGON ((-110.99899 36.58611, -111.00061 36.5..."
1,83302,275,"POLYGON ((-111.11099 36.69969, -111.11049 36.6..."
2,83359,332,"POLYGON ((-110.93996 36.75837, -110.94202 36.7..."
3,83364,337,"POLYGON ((-111.20558 36.84119, -111.20403 36.8..."
4,83366,339,"POLYGON ((-111.22911 36.84524, -111.22968 36.8..."
...,...,...,...
3846,86611,3556,"POLYGON ((-110.99458 40.41116, -110.99353 40.4..."
3847,86615,3560,"POLYGON ((-110.99661 40.49742, -110.99808 40.4..."
3848,86686,3631,"POLYGON ((-110.74581 39.88584, -110.74506 39.8..."
3849,86691,3636,"POLYGON ((-110.87315 39.87450, -110.87141 39.8..."


## Step 2: Sort the DataFrame by 'nhm_id'
Looks like data are sorted already but we'll sort anyway. CBH data supplied to this model should be sorted by nhm_id.

In [9]:
gdf_sorted = gdf.sort_values(by="nhm_id")

## Step 3: Export "nhm_id" to a file as read from the .param file.

In [10]:
nhm_id = read_param_values(root_dir, "nhm_id")
df = pd.DataFrame(nhm_id.astype(int))
df.to_csv(f'{root_dir}/nhm_id', index=False, header=False)

Read 3851 nhm_id values


## Step 4: Export "hru_lat.csv" "hru_lon.csv" to a file as read from the .param file.
These are used when converting hru results as .csv file to netcdf output.  The lat/lon give some spatial context to the
netcdf output files.

In [11]:
hru_lat = read_param_values(root_dir, "hru_lat")
df = pd.DataFrame(hru_lat)
df.to_csv(f'{root_dir}/hru_lat.csv', index=False, header=False)

hru_lon = read_param_values(root_dir, "hru_lon")
df = pd.DataFrame(hru_lon)
df.to_csv(f'{root_dir}/hru_lon.csv', index=False, header=False)

Read 3851 hru_lat values
Read 3851 hru_lon values


## 5 Get elevations - used for calculating humidity from cfsv2 data.

In [12]:

elevations = read_param_values(root_dir, "hru_elev")
df = pd.DataFrame(elevations)
df.to_csv(f'{root_dir}elevation.csv', index=False, header=False)

Read 3851 hru_elev values


## 6 Get segment lat/lon.  
The param db does not contain hru_lon generally and not hru_lat if there is no stream temp solved.  So rather than
using the db to extract those values we need to get them from the geometry file.

In [13]:
gdf = gpd.read_file(root_dir + "GIS/model_nsegment.shp")
gdf

,nhm_seg,model_seg_,geometry
0,42706,1,"LINESTRING (-840513.813 1962765.750, -840463.4..."
1,42707,2,"LINESTRING (-893201.427 1985197.890, -893186.9..."
2,42708,3,"LINESTRING (-855004.607 1970874.926, -855042.8..."
3,42709,4,"LINESTRING (-865168.318 1972729.691, -865089.6..."
4,42710,5,"LINESTRING (-881519.930 1968345.969, -881533.8..."
...,...,...,...
1937,44658,1938,"LINESTRING (-1202301.007 1565413.836, -1202326..."
1938,44659,1939,"LINESTRING (-1206408.194 1562910.585, -1206329..."
1939,44660,1940,"LINESTRING (-1206307.558 1541303.007, -1206322..."
1940,44661,1941,"LINESTRING (-1198807.062 1540005.963, -1198864..."


In [14]:
invalid_geometries = gdf[~gdf.is_valid]
print(f"Found {len(invalid_geometries)} invalid geometries.")

Found 0 invalid geometries.


In [15]:
gdf_sorted = gdf.sort_values(by="nhm_seg")
gdf_sorted['nhm_seg'].to_csv(
    f'{root_dir}nhm_seg', index=False, header=False
)
centroids = gdf_sorted.to_crs(5070).geometry.centroid
centroids_ll = centroids.to_crs(4326)
lats = centroids_ll.y
lons = centroids_ll.x
lats.to_csv(f'{root_dir}seg_lat.csv', index=False, header=False)
lons.to_csv(f'{root_dir}seg_lon.csv', index=False, header=False)

## Part 7 Generate gridmet weights.

In [16]:
from gdptools import WeightGen
from gdptools import AggGen
from gdptools import ClimRCatData


In [17]:
gdf = gpd.read_file(root_dir + "GIS/model_nhru.shp")
gdf

,nhm_id,model_hru_,geometry
0,83301,274,"POLYGON ((-1324125.133 1609094.887, -1324275.0..."
1,83302,275,"POLYGON ((-1331914.568 1623225.112, -1331901.0..."
2,83359,332,"POLYGON ((-1308375.036 1622938.630, -1308465.2..."
3,83364,337,"POLYGON ((-1337655.210 1640205.133, -1337535.3..."
4,83366,339,"POLYGON ((-1339634.862 1640985.080, -1339695.0..."
...,...,...,...
3846,86611,3556,"POLYGON ((-1256384.842 2032365.123, -1256295.1..."
3847,86615,3560,"POLYGON ((-1255034.951 2041934.777, -1255154.7..."
3848,86686,3631,"POLYGON ((-1244805.129 1970954.938, -1244744.9..."
3849,86691,3636,"POLYGON ((-1255664.885 1971375.143, -1255544.7..."


In [18]:
climater_cat = "https://github.com/mikejohnson51/climateR-catalogs/releases/download/March-2024/catalog.parquet"
cat = pd.read_parquet(climater_cat)
cat.head()

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
0,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp45_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
1,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_ACCESS1-0_rcp85_r1i1p1,pr,Precipitation,mm/d,ACCESS1-0,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
2,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp26_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
3,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp45_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX
4,bcca,None,https://cida.usgs.gov/thredds/dodsC/cmip5_bcca...,opendap,BCCA_0-125deg_pr_day_bcc-csm1-1_rcp60_r1i1p1,pr,Precipitation,mm/d,bcc-csm1-1,r1i1p1,...,25.1875,52.8125,0.125,0.125,462.0,222.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,True,T,TYX


In [19]:
_id = "gridmet"
_varname = "tmmx"

# an example query returns a pandas dataframe.
tc = cat.query(
    "id == @_id & variable == @_varname"
)
tc

,id,asset,URL,type,varname,variable,description,units,model,ensemble,...,Y1,Yn,resX,resY,ncols,nrows,crs,toptobottom,tiled,dim_order
13997,gridmet,None,http://thredds.northwestknowledge.net:8080/thr...,opendap,daily_maximum_temperature,tmmx,tmmx,K,None,None,...,49.4,25.066667,0.041667,0.041667,1386.0,585.0,+proj=longlat +a=6378137 +f=0.0033528106647474...,False,,TYX


In [20]:
# Create a dictionary of parameter dataframes for each variable
tvars = ["tmmx"]
cat_params = [cat.query("id == @_id & variable == @_var").to_dict(orient="records")[0] for _var in tvars]

cat_dict = dict(zip(tvars, cat_params))

# Output an example of the cat_param.json entry for "aet".
cat_dict.get("tmmx")

{'id': 'gridmet',
 'asset': None,
 'URL': 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_tmmx_1979_CurrentYear_CONUS.nc',
 'type': 'opendap',
 'varname': 'daily_maximum_temperature',
 'variable': 'tmmx',
 'description': 'tmmx',
 'units': 'K',
 'model': None,
 'ensemble': None,
 'scenario': None,
 'T_name': 'day',
 'duration': '1979-01-01/..',
 'interval': '1 days',
 'nT': nan,
 'X_name': 'lon',
 'Y_name': 'lat',
 'X1': -124.76666663333334,
 'Xn': -67.05833330000002,
 'Y1': 49.400000000000006,
 'Yn': 25.066666666666666,
 'resX': 0.041666666666666664,
 'resY': 0.04166666666666668,
 'ncols': 1386.0,
 'nrows': 585.0,
 'crs': '+proj=longlat +a=6378137 +f=0.00335281066474748 +pm=0 +no_defs',
 'toptobottom': False,
 'tiled': '',
 'dim_order': 'TYX'}

In [21]:
user_data = ClimRCatData(
    cat_dict=cat_dict,
    f_feature=gdf,
    id_feature='nhm_id',
    period=["1980-01-01", "1980-12-31"]
)

wght_gen = WeightGen(
    user_data=user_data,
    method="serial",
    output_file=root_dir + "GIS/uc_weights.csv",
    weight_gen_crs=6931
)

wghts = wght_gen.calculate_weights(intersections=False)

Using serial engine
grid cells generated in 4.8607 seconds
Data preparation finished in 5.7525 seconds
     - validating target polygons
     - fixing 177 invalid polygons.
     - validating source polygons
     - fixing 0 invalid polygons.
Validate polygons finished in 0.2675 seconds
     - reprojecting and validating source polygons
     - checking the source polygons for invalid polygons
     - checking source for empty polygons
     - reprojecting and validating target polygons
     - checking the target polygons for invalid polygons
     - checking target for empty polygons
Reprojecting to: EPSG:6931 and validating polygons finished in 0.5 seconds
Intersections finished in 4.6753 seconds
Weight gen finished in 4.6809 seconds


In [7]:
import os
import subprocess
prms_path = '/home/rmcd/prms_5.2.1_linux/bin/prms_5_2_1_1'
project_root = '/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1'
project_output = Path('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output')
cbh_path = Path("/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original")
# hard-coded by looking at bandit.control file 
start_time = "1979-01-01"
end_time = "2021-12-31"
os.chdir(project_root)
print(os.getcwd())
command = [
   prms_path,
   "-set", "start_time", start_time,
   "-set", "end_time", end_time,
   "-set", "humidity_day", cbh_path / "humidity.day",
   "-set", "precip_day", cbh_path / "precip.day",
   "-set", "tmax_day", cbh_path / "tmax.day",
   "-set", "tmin_day", cbh_path / "tmin.day",
   "-set", "save_vars_to_file", "1",
   "-set", "var_save_file", f"{project_root}/daily/restart/{end_time}.restart",
   "-set", "nhruOutBaseFileName", project_output,
   "-set", "nsegmentOutBaseFileName", project_output,
   "-set", "model_output_file", project_output / "model.out",
   "-set", "param_file", f"{project_root}/myparam.param",
   "-C", f"{project_root}/NHM_UC.control"
]
subprocess.run(command, check=True)

/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1



                         U.S. Geological Survey
               Precipitation-Runoff Modeling System (PRMS)
                        Version 5.2.1.1 10/04/2023

        Process            Available Modules
--------------------------------------------------------------------
  Basin Definition: basin
    Cascading Flow: cascade
  Time Series Data: obs, water_use_read, dynamic_param_read
   Potet Solar Rad: soltab
  Temperature Dist: temp_1sta, temp_laps, temp_dist2, climate_hru,
                    temp_map
       Precip Dist: precip_1sta, precip_laps, precip_dist2,
                    climate_hru, precip_map
Temp & Precip Dist: xyz_dist, ide_dist
    Solar Rad Dist: ccsolrad, ddsolrad, climate_hru
Transpiration Dist: transp_tindex, climate_hru, transp_frost
      Potential ET: potet_hamon, potet_jh, potet_pan, climate_hru,
                    potet_hs, potet_pt, potet_pm, potet_pm_sta
      Interception: intcp
Snow & Glacr Dynam: snowcomp, glacr_melt
   


Control variable start_time set to 1979-01-01.


Control variable end_time set to 2021-12-31.


Control variable humidity_day set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/humidity.day.


Control variable precip_day set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/precip.day.


Control variable tmax_day set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/tmax.day.


Control variable tmin_day set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/tmin.day.


Control variable save_vars_to_file set to 1.


Control variable var_save_file set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/restart/2021-12-31.restart.


Control variable nhruOutBaseFileName set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output.


Control variable nsegmentOutBaseFileName set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output.


Control variable model_output_file set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output/model.out.


Control variable param_file set to /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/myparam.param.







Using Control File: /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/NHM_UC.control

Using Parameter File: /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/myparam.param

Writing var_save_file: /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/restart/2021-12-31.restart

Writing PRMS Water Budget File: /mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output/model.out



         set in the Parameter File. Module default values are being used.

         set in the Parameter File. Module default values are being used.

         set in the Parameter File. Module default values are being used.

         set in the Parameter File. Module default values are being used.


 WARNING, seg_slope < 0.0000001, set to 0.0000001        1109   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0000001        1109   0.00000000    
 WARNING, seg_close not specified so setting to segment_up or other segment if no segment_up
 WARNING, segment_up = 0 without associated HRU for segment:         162
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1344
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1381
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1396
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1935
          will use previous segment in route order to set segment values

Simulation time pe

CompletedProcess(args=['/home/rmcd/prms_5.2.1_linux/bin/prms_5_2_1_1', '-set', 'start_time', '1979-01-01', '-set', 'end_time', '2021-12-31', '-set', 'humidity_day', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/humidity.day'), '-set', 'precip_day', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/precip.day'), '-set', 'tmax_day', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/tmax.day'), '-set', 'tmin_day', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1_original/tmin.day'), '-set', 'save_vars_to_file', '1', '-set', 'var_save_file', '/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/restart/2021-12-31.restart', '-set', 'nhruOutBaseFileName', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output'), '-set', 'nsegmentOutBaseFileName', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output'), '-set', 'model_output_file', PosixPath('/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/daily/output/model.out'), '-set', 'param_file', '/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/myparam.param', '-C', '/mnt/d/oNHM/NHM_PRMS_UC_GF_1_1/NH

In [14]:
import os
import subprocess
prms_path = '/home/rmcd/prms_5.2.1_linux/bin/prms_5_2_1_1'
project_root = '/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1'
project_output = Path('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/output')
cbh_path = Path("/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input")
# hard-coded by looking at bandit.control file 
start_date = "2023-01-01"
end_date = "2025-3-10"
start_time = "2023,1,1,0,0,0"
end_time = "2025,3,10,0,0,0"
restart_time = "2022-12-31"
os.chdir(project_root)
command = [
   prms_path,
   "-set", "start_time", start_time,
   "-set", "end_time", end_time,
   "-set", "humidity_day", cbh_path / "humidity.cbh",
   "-set", "precip_day", cbh_path / "prcp.cbh",
   "-set", "tmax_day", cbh_path / "tmax.cbh",
   "-set", "tmin_day", cbh_path / "tmin.cbh",
   "-set", "init_vars_from_file", "1",
   "-set", "var_init_file", f"{project_root}/daily/restart/{restart_time}.restart",
   "-set", "save_vars_to_file", "1",
   "-set", "var_save_file", f"{project_root}/daily/restart/{end_date}.restart",
   "-set", "nhruOutBaseFileName", project_output,
   "-set", "nsegmentOutBaseFileName", project_output,
   "-set", "model_output_file", project_output / "model.out",
   "-set", "param_file", f"{project_root}/myparam.param",
   "-C", f"{project_root}/NHM_UC.control"
]
subprocess.run(command, check=True)


Control variable start_time set to 2023,1,1,0,0,0.


Control variable end_time set to 2025,3,10,0,0,0.


Control variable humidity_day set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/humidity.cbh.


Control variable precip_day set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/prcp.cbh.


Control variable tmax_day set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/tmax.cbh.


Control variable tmin_day set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/tmin.cbh.


Control variable init_vars_from_file set to 1.


Control variable var_init_file set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/restart/2022-12-31.restart.


Control variable save_vars_to_file set to 1.


Control variable var_save_file set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/restart/2025-3-10.restart.


Control variable nhruOutBaseFileName set to /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/output.


Control variable nsegmentOutBaseFileName set to /home/rmcd/nhm-bind/NHM_PRMS_UC_




                         U.S. Geological Survey
               Precipitation-Runoff Modeling System (PRMS)
                        Version 5.2.1.1 10/04/2023

        Process            Available Modules
--------------------------------------------------------------------
  Basin Definition: basin
    Cascading Flow: cascade
  Time Series Data: obs, water_use_read, dynamic_param_read
   Potet Solar Rad: soltab
  Temperature Dist: temp_1sta, temp_laps, temp_dist2, climate_hru,
                    temp_map
       Precip Dist: precip_1sta, precip_laps, precip_dist2,
                    climate_hru, precip_map
Temp & Precip Dist: xyz_dist, ide_dist
    Solar Rad Dist: ccsolrad, ddsolrad, climate_hru
Transpiration Dist: transp_tindex, climate_hru, transp_frost
      Potential ET: potet_hamon, potet_jh, potet_pan, climate_hru,
                    potet_hs, potet_pt, potet_pm, potet_pm_sta
      Interception: intcp
Snow & Glacr Dynam: snowcomp, glacr_melt
    Surface Runoff: srunoff_smidx,


Using Control File: /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/NHM_UC.control

Using Parameter File: /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/myparam.param

Using var_init_file: /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/restart/2022-12-31.restart

Writing var_save_file: /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/restart/2025-3-10.restart

Writing PRMS Water Budget File: /home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/output/model.out



         set in the Parameter File. Module default values are being used.

         set in the Parameter File. Module default values are being used.

         set in the Parameter File. Module default values are being used.

         set in the Parameter File. Module default values are being used.


 WARNING, seg_slope < 0.0000001, set to 0.0000001        1109   0.00000000    
 WARNING, seg_slope < 0.0000001, set to 0.0000001        1109   0.00000000    
 WARNING, seg_close not specified so setting to segment_up or other segment if no segment_up
 WARNING, segment_up = 0 without associated HRU for segment:         162
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1344
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1381
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1396
          will use previous segment in route order to set segment values
 WARNING, segment_up = 0 without associated HRU for segment:        1935
          will use previous segment in route order to set segment values

Simulation time pe

read_line:  coming out of storm. dt = 1 day
ERROR STOP 10


CalledProcessError: Command '['/home/rmcd/prms_5.2.1_linux/bin/prms_5_2_1_1', '-set', 'start_time', '2023,1,1,0,0,0', '-set', 'end_time', '2025,3,10,0,0,0', '-set', 'humidity_day', PosixPath('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/humidity.cbh'), '-set', 'precip_day', PosixPath('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/prcp.cbh'), '-set', 'tmax_day', PosixPath('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/tmax.cbh'), '-set', 'tmin_day', PosixPath('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/input/tmin.cbh'), '-set', 'init_vars_from_file', '1', '-set', 'var_init_file', '/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/restart/2022-12-31.restart', '-set', 'save_vars_to_file', '1', '-set', 'var_save_file', '/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/restart/2025-3-10.restart', '-set', 'nhruOutBaseFileName', PosixPath('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/output'), '-set', 'nsegmentOutBaseFileName', PosixPath('/home/rmcd/nhm-bind/NHM_PRMS_UC_GF_1_1/daily/